In [1]:
# Import Required Python Libraries

import tkinter as tkinter
from tkinter import *
from PIL import Image, ImageTk
from tkinter import filedialog
from tkinter import Tk, Label, Canvas
from tkinter import ttk
import pandas as pd
import urllib
import requests
import numpy as np

# Business Logic

In [2]:
# This function gets values from user inputs and applies Business Logic on the code
def send(day, time1, time2):
    
#   Get user inputs in local variables
    day = day
    time1 = time1
    time2 = time2
    
#   Append the user input values to the API URL
    url='http://192.168.1.116:8080/nyc-busdata/webapi/transport/details/'+day+'/'+time1+'/'+time2
    print(url)
    
#   Call API to get values
    response = requests.get(url)
    
#   Get data from JSON returned by the API
    data = response.json()
    
#   Get JSON data in a DataFrame
    importeddf = pd.DataFrame(data)


#   Change Datatype of Expected Time of Arrival from String to Datetime
    importeddf["expectedArrivalTime"] = pd.to_datetime(importeddf["expectedArrivalTime"],format= '%H:%M:%S')

#   Change Datatype of Scheduled Time of Arrival from String to Datetime
    importeddf["scheduledArrivalTime"] = pd.to_datetime(importeddf["scheduledArrivalTime"],format= '%H:%M:%S')

#   Add Calculated Column of 'Delay'
    importeddf["Delay"] = importeddf["expectedArrivalTime"] - importeddf["scheduledArrivalTime"]

#   Change Datatype of Delay to numeric
    importeddf["Delay"] = pd.to_numeric(importeddf["Delay"])

#   Convert Nanoseconds to Minutes
    importeddf["DelayInMinutes"] = importeddf["Delay"]/60000000000 
    
#   Round Delay in minutes and add to a new Dataframe
    dataframe = pd.DataFrame(importeddf.round({'DelayInMinutes':1}))


#   Groupby NextStop and take Mean of Delays 
    groupeddataframe = pd.DataFrame(dataframe.groupby(["nextStopPointName"]).mean())

    ascendingDF = pd.DataFrame(groupeddataframe.sort_values(["DelayInMinutes"],ascending=False))

#   Adding Column to determine if there is an average delay or not
    ascendingDF.loc[ascendingDF.DelayInMinutes > 0, 'Delay?'] = 'True' 

    OnlyPoditiveDF = pd.DataFrame(ascendingDF[ascendingDF['Delay?'] == 'True'])

#   Get only positive Delay values in the final output
    finalDF = pd.DataFrame(OnlyPoditiveDF['DelayInMinutes'])

#   Returns 'finalDF' DataFrame to the Front End Code
    return(finalDF.round({'DelayInMinutes':1}))

# Front End

In [3]:
# Create master window dialog box
master = tkinter.Tk()

# Set background color
master.configure(background='cyan3')

# Set title
master.title("Taxi Mate")

# Function which gets values from user inputs and sends them to the business logic through "send" function
def values_return(event):
    content1 = entry1.get()
    content2 = entry2.get()
    content3 = entry3.get()
    output.delete('1.0', END)
    
#   Call business logic function "send" and print output in GUI
    output.insert(END, (str(send(content1, content2, content3)) + '\n'))

    
# Create Labels
Label(master, text="Day: ", bg='cyan3', font='Helvetica 18 bold', fg = 'white').grid(row=0, sticky=W)
Label(master, text="Time1: ", bg = 'cyan3', font='Helvetica 18 bold', fg = 'white').grid(row=1, sticky=W)
Label(master, text="Time2: ", bg = 'cyan3', font='Helvetica 18 bold', fg = 'white').grid(row=2, sticky=W)

# Create Entries to take user inputs
entry1 = Entry(master, bg = 'snow3', borderwidth = 3, relief = RAISED)
entry2 = Entry(master, bg = 'snow3', borderwidth = 3, relief = RAISED)
entry3 = Entry(master, bg = 'snow3', borderwidth = 3, relief = RAISED)

# Create a Submit Button
submit = Button(master, text="Submit", bg = 'SlateGray3', fg = 'white', font='Helvetica 11 bold', borderwidth = 3, relief = RAISED)

# Set postions of Entries on the GUI Window
entry1.grid(row=0, column=1)
entry2.grid(row=1, column=1)
entry3.grid(row=2, column=1)

# Set position of Submit Button on the GUI Window
submit.grid(row=3, column=1)

# Set Output Box postion and size
output = Text(master, width=120, height= 30, bg = 'snow3', borderwidth = 3, relief = RIDGE)
output.grid(row=4, column=1)

# Bind Submit Button to an event which calls the "values_return" funtion to get user inputs
submit.bind('<Button>', values_return)

# Infinite Loop to keep the GUI window runnning
mainloop()

http://192.168.1.116:8080/nyc-busdata/webapi/transport/details/Thursday/00:30:00/00:40:00
